<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Copy_of_rag_fusion_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Fusion Query Pipeline

<a href="https://colab.research.google.com/github/run-llama/llama-hub/blob/main/llama_hub/llama_packs/query/rag_fusion_pipeline/rag_fusion_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows how to implement RAG Fusion using the LlamaIndex Query Pipeline syntax.

## Setup / Load Data

We load in the pg_essay.txt data.

In [ ]:
#added by Frank Morales(FM) 11/01/2024
%pip install openai  --root-user-action=ignore
!pip install llama_index phoenix pyvis network
!pip install llama_hub
%pip install colab-env --upgrade --quiet --root-user-action=ignore

import colab_env
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt' -O pg_essay.txt

In [2]:
#%pip install openai --root-user-action=ignore

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
#print(os.getenv("OPENAI_API_KEY"))

from llama_index import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_files=["pg_essay.txt"])
docs = reader.load_data()

### [Optional] Setup Tracing

We also setup tracing through Arize Phoenix to look at our outputs.

In [3]:
#import phoenix as px

#px.launch_app()
#import llama_index

#llama_index.set_global_handler("arize_phoenix")

## Setup Llama Pack

Next we download the LlamaPack. All the code is in the downloaded directory - we encourage you to take a look to see the QueryPipeline syntax!

In [4]:
# Option 1: Use `download_llama_pack`
# from llama_index.llama_pack import download_llama_pack

# RAGFusionPipelinePack = download_llama_pack(
#     "RAGFusionPipelinePack",
#     "./rag_fusion_pipeline_pack",
#     # leave the below line commented out if using the notebook on main
#     # llama_hub_url="https://raw.githubusercontent.com/run-llama/llama-hub/jerry/add_query_pipeline_pack/llama_hub"
# )

# Option 2: Import from llama_hub package
from llama_hub.llama_packs.query.rag_fusion_pipeline.base import RAGFusionPipelinePack


from llama_index.llms import OpenAI

In [5]:
pack = RAGFusionPipelinePack(docs, llm=OpenAI(model="gpt-3.5-turbo"))

## Inspecting the Code

If we take a look at how it's setup (in your downloaded directory, you'll see the following code using our QueryPipeline syntax).

`retrievers` is a dictionary mapping a chunk size to retrievers (chunk sizes: 128, 256, 512, 1024).

```python
# construct query pipeline
p = QueryPipeline()
module_dict = {
    **self.retrievers,
    "input": InputComponent(),
    "summarizer": TreeSummarize(),
    # NOTE: Join args
    "join": ArgPackComponent(),
    "reranker": rerank_component,
}
p.add_modules(module_dict)
# add links from input to retriever (id'ed by chunk_size)
for chunk_size in self.chunk_sizes:
    p.add_link("input", str(chunk_size))
    p.add_link(str(chunk_size), "join", dest_key=str(chunk_size))
p.add_link("join", "reranker")
p.add_link("input", "summarizer", dest_key="query_str")
p.add_link("reranker", "summarizer", dest_key="nodes")
```

We visualize the DAG below.

In [6]:
from pyvis.network import Network

net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(pack.query_pipeline.dag)
net.show("rag_dag.html")

rag_dag.html


In [21]:
#modify By FM 11/01/2024

#response = pack.run(query="What did the author do growing up?")
query0="What did the author do growing up?"
query='I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.'
query1 = "Who is the President of the USA?"
query2 = "Who won the baseball World Series in 2020? and Who Lost"
query3 = 'Anything about FORTRAN'
query4 = 'Anything about LIPS'
query5 = 'Anything about Python'

response0 = pack.run(query=query0)
response1 = pack.run(query=query1)
response2 = pack.run(query=query2)
response4 = pack.run(query=query4)

print()
print(query0)
print(str(response0))
print()

print()
print(query1)
print(str(response1))
print()

print()
print(query2)
print(str(response2))
print()

print()
print(query4)
print(str(response4))
print()


What did the author do growing up?
The author worked on writing and programming growing up. They wrote short stories and tried writing programs on an IBM 1401 computer.


Who is the President of the USA?
I'm sorry, but I cannot answer that question based on the given context information.


Who won the baseball World Series in 2020? and Who Lost
I'm sorry, but I cannot answer that query based on the given context information.


Anything about LIPS
Lisp is a programming language that was originally intended as a formal model of computation, an alternative to the Turing machine. It was invented by John McCarthy and later became a programming language in the ordinary sense. Lisp has a core that is defined by writing an interpreter in itself. McCarthy's original Lisp interpreter could only interpret Lisp expressions and lacked many features of a programming language. Over time, these features were added, but not using McCarthy's original axiomatic approach. Lisp has a power and elegance th

In [8]:
#response.source_nodes